# 0.0 - Importando as bibliotecas

In [2]:
import pandas         as pd
import seaborn        as sns
import plotly.express as px
import ipywidgets     as widgets

from ipywidgets            import fixed
from matplotlib            import gridspec
from matplotlib            import pyplot as plt
from geopy.geocoders       import Nominatim
from IPython.core.display  import HTML
from IPython.display       import Image


## 0.1 - Funções

In [3]:
def jupyter_settings():
    
    %matplotlib inline 
    #%pylab inline
    
    plt.style.use('bmh')
    plt.rcParams['figure.figsize']=[20,10]
    plt.rcParams['font.size']=10
    
    display( HTML('<style>.container {width:100% !important; }</style>'))
    pd.options.display.max_columns = None
    pd.options.display.max_rows = None
    pd.set_option('display.expand_frame_repr',False )
    pd.set_option('display.float_format',lambda x: '%.2f' % x)
    

    sns.set()
    


jupyter_settings()

## 0.2 - Carregando os dados

In [4]:
df_raw = pd.read_csv('../data/kc_house_data.csv')

In [8]:
df = df_raw.copy()

In [9]:
df.head(5)

,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
0,7129300520,20141013T000000,221900.00,3,1.00,1180,5650,1.00,0,0,3,7,1180,0,1955,0,98178,47.51,-122.26,1340,5650
1,6414100192,20141209T000000,538000.00,3,2.25,2570,7242,2.00,0,0,3,7,2170,400,1951,1991,98125,47.72,-122.32,1690,7639
2,5631500400,20150225T000000,180000.00,2,1.00,770,10000,1.00,0,0,3,6,770,0,1933,0,98028,47.74,-122.23,2720,8062
3,2487200875,20141209T000000,604000.00,4,3.00,1960,5000,1.00,0,0,5,7,1050,910,1965,0,98136,47.52,-122.39,1360,5000
4,1954400510,20150218T000000,510000.00,3,2.00,1680,8080,1.00,0,0,3,8,1680,0,1987,0,98074,47.62,-122.05,1800,7503


# 1.0 -  Qual a média do preço de compra dos imóveis por "Nível"?

- Nível 0 -> Preço entre  0 e 321.950
- Nível 1 -> Preço entre 321.950 e  450.000
- Nível 2 -> Preço entre 450.000 e 645.000
- Nível 3 -> Acima de 645.000

In [10]:
df['level']='NA'

for i in range(len(df)):
    
    if (df.loc[i,'price'] > 0) & (df.loc[i,'price'] <321950):
        df.loc[i,'level'] = '0'
    
    elif (df.loc[i,'price'] >= 321950) & (df.loc[i,'price'] < 450000):
        df.loc[i,'level'] = '1'
    
    elif (df.loc[i,'price'] >= 450000) & (df.loc[i,'price'] < 650000):
        df.loc[i,'level'] = '2'
     
    else:
        df.loc[i,'level'] = '3'

df['level'] = df['level'].astype(int)

# 2.0 - Qual a média do tamanho da sala de estar dos imóveis por "Size"?

- Size 0 -> Tamanho entre 0 e 1427 sqft
- Size 1 -> Tamanho entre 1427 e 1910 sqft
- Size 2 -> Tamanho entre 1910 e 2550 sqft
- Size 3 -> Tamanho acima de 2550 sqft

In [11]:
df['size']= 'NA'


df['size']= df['sqft_living'].apply(lambda x: 0  if x < 1427 else 
                                              1  if (x >= 1427) & (x < 1910) else 
                                              2  if (x >= 1910) & (x < 2550) else 3)  



df['size'] = df['size'].astype(int)

# 3.0 - Adicione as seguintes informações ao conjunto de dados original:
- PlaceID: identificação da localização.
- OSM Type: Open Street Map Type
- Country: Nome do País
- Country Code: Código do país

In [7]:
df1 = df.copy()

In [9]:
# Create empty rows
df1['country'] = 'NA'
df1['state'] = 'NA'
df1['city'] = 'NA'
df1['neighbourhood'] = 'NA'
df1['road'] = 'NA'
df1['house_number'] = 'NA'

# Initialize Nominatim API
geolocator = Nominatim(user_agent='geoapiExercises')

for i in range(len(df1)):
    
    print('Loop:{} / {}'.format(i,len(df1)))
    
    query = str(df1.loc[i,'lat']) + ','+ str(df1.loc[i,'long'])
    
    #API request
    response = geolocator.reverse(query)
    
    # Populate data
    if 'country' in response.raw['address']:
        df1.loc[i,'country'] = response.raw['address']['country']
           
    if 'state' in response.raw['address']:
        df1.loc[i,'state'] = response.raw['address']['state']
    
    if 'city' in response.raw['address']:
        df1.loc[i,'city'] = response.raw['address']['city']
    
    if 'neighbourhood' in response.raw['address']:
        df1.loc[i,'neighbourhood'] = response.raw['address']['neighbourhood']
    
    if 'road' in response.raw['address']:
        df1.loc[i,'road'] = response.raw['address']['road']
    
    if 'house_number' in response.raw['address']:
        df1.loc[i,'house_number'] = response.raw['address']['house_number']

Loop:0 / 21613
Loop:1 / 21613
Loop:2 / 21613
Loop:3 / 21613
Loop:4 / 21613
Loop:5 / 21613
Loop:6 / 21613
Loop:7 / 21613
Loop:8 / 21613
Loop:9 / 21613
Loop:10 / 21613
Loop:11 / 21613
Loop:12 / 21613
Loop:13 / 21613
Loop:14 / 21613
Loop:15 / 21613
Loop:16 / 21613
Loop:17 / 21613
Loop:18 / 21613
Loop:19 / 21613
Loop:20 / 21613
Loop:21 / 21613
Loop:22 / 21613
Loop:23 / 21613
Loop:24 / 21613
Loop:25 / 21613
Loop:26 / 21613
Loop:27 / 21613
Loop:28 / 21613
Loop:29 / 21613
Loop:30 / 21613
Loop:31 / 21613
Loop:32 / 21613
Loop:33 / 21613
Loop:34 / 21613
Loop:35 / 21613
Loop:36 / 21613
Loop:37 / 21613
Loop:38 / 21613
Loop:39 / 21613
Loop:40 / 21613
Loop:41 / 21613
Loop:42 / 21613
Loop:43 / 21613
Loop:44 / 21613
Loop:45 / 21613
Loop:46 / 21613
Loop:47 / 21613
Loop:48 / 21613
Loop:49 / 21613
Loop:50 / 21613
Loop:51 / 21613
Loop:52 / 21613
Loop:53 / 21613
Loop:54 / 21613
Loop:55 / 21613
Loop:56 / 21613
Loop:57 / 21613
Loop:58 / 21613
Loop:59 / 21613
Loop:60 / 21613
Loop:61 / 21613
Loop:62 / 21613
Lo

Loop:489 / 21613
Loop:490 / 21613
Loop:491 / 21613
Loop:492 / 21613
Loop:493 / 21613
Loop:494 / 21613
Loop:495 / 21613
Loop:496 / 21613
Loop:497 / 21613
Loop:498 / 21613
Loop:499 / 21613
Loop:500 / 21613
Loop:501 / 21613
Loop:502 / 21613
Loop:503 / 21613
Loop:504 / 21613
Loop:505 / 21613
Loop:506 / 21613
Loop:507 / 21613
Loop:508 / 21613
Loop:509 / 21613
Loop:510 / 21613
Loop:511 / 21613
Loop:512 / 21613
Loop:513 / 21613
Loop:514 / 21613
Loop:515 / 21613
Loop:516 / 21613
Loop:517 / 21613
Loop:518 / 21613
Loop:519 / 21613
Loop:520 / 21613
Loop:521 / 21613
Loop:522 / 21613
Loop:523 / 21613
Loop:524 / 21613
Loop:525 / 21613
Loop:526 / 21613
Loop:527 / 21613
Loop:528 / 21613
Loop:529 / 21613
Loop:530 / 21613
Loop:531 / 21613
Loop:532 / 21613
Loop:533 / 21613
Loop:534 / 21613
Loop:535 / 21613
Loop:536 / 21613
Loop:537 / 21613
Loop:538 / 21613
Loop:539 / 21613
Loop:540 / 21613
Loop:541 / 21613
Loop:542 / 21613
Loop:543 / 21613
Loop:544 / 21613
Loop:545 / 21613
Loop:546 / 21613
Loop:547 / 216

Loop:971 / 21613
Loop:972 / 21613
Loop:973 / 21613
Loop:974 / 21613
Loop:975 / 21613
Loop:976 / 21613
Loop:977 / 21613
Loop:978 / 21613
Loop:979 / 21613
Loop:980 / 21613
Loop:981 / 21613
Loop:982 / 21613
Loop:983 / 21613
Loop:984 / 21613
Loop:985 / 21613
Loop:986 / 21613
Loop:987 / 21613
Loop:988 / 21613
Loop:989 / 21613
Loop:990 / 21613
Loop:991 / 21613
Loop:992 / 21613
Loop:993 / 21613
Loop:994 / 21613
Loop:995 / 21613
Loop:996 / 21613
Loop:997 / 21613
Loop:998 / 21613
Loop:999 / 21613
Loop:1000 / 21613
Loop:1001 / 21613
Loop:1002 / 21613
Loop:1003 / 21613
Loop:1004 / 21613
Loop:1005 / 21613
Loop:1006 / 21613
Loop:1007 / 21613
Loop:1008 / 21613
Loop:1009 / 21613
Loop:1010 / 21613
Loop:1011 / 21613
Loop:1012 / 21613
Loop:1013 / 21613
Loop:1014 / 21613
Loop:1015 / 21613
Loop:1016 / 21613
Loop:1017 / 21613
Loop:1018 / 21613
Loop:1019 / 21613
Loop:1020 / 21613
Loop:1021 / 21613
Loop:1022 / 21613
Loop:1023 / 21613
Loop:1024 / 21613
Loop:1025 / 21613
Loop:1026 / 21613
Loop:1027 / 21613
Loo

Loop:1428 / 21613
Loop:1429 / 21613
Loop:1430 / 21613
Loop:1431 / 21613
Loop:1432 / 21613
Loop:1433 / 21613
Loop:1434 / 21613
Loop:1435 / 21613
Loop:1436 / 21613
Loop:1437 / 21613
Loop:1438 / 21613
Loop:1439 / 21613
Loop:1440 / 21613
Loop:1441 / 21613
Loop:1442 / 21613
Loop:1443 / 21613
Loop:1444 / 21613
Loop:1445 / 21613
Loop:1446 / 21613
Loop:1447 / 21613
Loop:1448 / 21613
Loop:1449 / 21613
Loop:1450 / 21613
Loop:1451 / 21613
Loop:1452 / 21613
Loop:1453 / 21613
Loop:1454 / 21613
Loop:1455 / 21613
Loop:1456 / 21613
Loop:1457 / 21613
Loop:1458 / 21613
Loop:1459 / 21613
Loop:1460 / 21613
Loop:1461 / 21613
Loop:1462 / 21613
Loop:1463 / 21613
Loop:1464 / 21613
Loop:1465 / 21613
Loop:1466 / 21613
Loop:1467 / 21613
Loop:1468 / 21613
Loop:1469 / 21613
Loop:1470 / 21613
Loop:1471 / 21613
Loop:1472 / 21613
Loop:1473 / 21613
Loop:1474 / 21613
Loop:1475 / 21613
Loop:1476 / 21613
Loop:1477 / 21613
Loop:1478 / 21613
Loop:1479 / 21613
Loop:1480 / 21613
Loop:1481 / 21613
Loop:1482 / 21613
Loop:1483 

Loop:1884 / 21613
Loop:1885 / 21613
Loop:1886 / 21613
Loop:1887 / 21613
Loop:1888 / 21613
Loop:1889 / 21613
Loop:1890 / 21613
Loop:1891 / 21613
Loop:1892 / 21613
Loop:1893 / 21613
Loop:1894 / 21613
Loop:1895 / 21613
Loop:1896 / 21613
Loop:1897 / 21613
Loop:1898 / 21613
Loop:1899 / 21613
Loop:1900 / 21613
Loop:1901 / 21613
Loop:1902 / 21613
Loop:1903 / 21613
Loop:1904 / 21613
Loop:1905 / 21613
Loop:1906 / 21613
Loop:1907 / 21613
Loop:1908 / 21613
Loop:1909 / 21613
Loop:1910 / 21613
Loop:1911 / 21613
Loop:1912 / 21613
Loop:1913 / 21613
Loop:1914 / 21613
Loop:1915 / 21613
Loop:1916 / 21613
Loop:1917 / 21613
Loop:1918 / 21613
Loop:1919 / 21613
Loop:1920 / 21613
Loop:1921 / 21613
Loop:1922 / 21613
Loop:1923 / 21613
Loop:1924 / 21613
Loop:1925 / 21613
Loop:1926 / 21613
Loop:1927 / 21613
Loop:1928 / 21613
Loop:1929 / 21613
Loop:1930 / 21613
Loop:1931 / 21613
Loop:1932 / 21613
Loop:1933 / 21613
Loop:1934 / 21613
Loop:1935 / 21613
Loop:1936 / 21613
Loop:1937 / 21613
Loop:1938 / 21613
Loop:1939 

...

Loop:19591 / 21613
Loop:19592 / 21613
Loop:19593 / 21613
Loop:19594 / 21613
Loop:19595 / 21613
Loop:19596 / 21613
Loop:19597 / 21613
Loop:19598 / 21613
Loop:19599 / 21613
Loop:19600 / 21613
Loop:19601 / 21613
Loop:19602 / 21613
Loop:19603 / 21613
Loop:19604 / 21613
Loop:19605 / 21613
Loop:19606 / 21613
Loop:19607 / 21613
Loop:19608 / 21613
Loop:19609 / 21613
Loop:19610 / 21613
Loop:19611 / 21613
Loop:19612 / 21613
Loop:19613 / 21613
Loop:19614 / 21613
Loop:19615 / 21613
Loop:19616 / 21613
Loop:19617 / 21613
Loop:19618 / 21613
Loop:19619 / 21613
Loop:19620 / 21613
Loop:19621 / 21613
Loop:19622 / 21613
Loop:19623 / 21613
Loop:19624 / 21613
Loop:19625 / 21613
Loop:19626 / 21613
Loop:19627 / 21613
Loop:19628 / 21613
Loop:19629 / 21613
Loop:19630 / 21613
Loop:19631 / 21613
Loop:19632 / 21613
Loop:19633 / 21613
Loop:19634 / 21613
Loop:19635 / 21613
Loop:19636 / 21613
Loop:19637 / 21613
Loop:19638 / 21613
Loop:19639 / 21613
Loop:19640 / 21613
Loop:19641 / 21613
Loop:19642 / 21613
Loop:19643 /

Loop:20023 / 21613
Loop:20024 / 21613
Loop:20025 / 21613
Loop:20026 / 21613
Loop:20027 / 21613
Loop:20028 / 21613
Loop:20029 / 21613
Loop:20030 / 21613
Loop:20031 / 21613
Loop:20032 / 21613
Loop:20033 / 21613
Loop:20034 / 21613
Loop:20035 / 21613
Loop:20036 / 21613
Loop:20037 / 21613
Loop:20038 / 21613
Loop:20039 / 21613
Loop:20040 / 21613
Loop:20041 / 21613
Loop:20042 / 21613
Loop:20043 / 21613
Loop:20044 / 21613
Loop:20045 / 21613
Loop:20046 / 21613
Loop:20047 / 21613
Loop:20048 / 21613
Loop:20049 / 21613
Loop:20050 / 21613
Loop:20051 / 21613
Loop:20052 / 21613
Loop:20053 / 21613
Loop:20054 / 21613
Loop:20055 / 21613
Loop:20056 / 21613
Loop:20057 / 21613
Loop:20058 / 21613
Loop:20059 / 21613
Loop:20060 / 21613
Loop:20061 / 21613
Loop:20062 / 21613
Loop:20063 / 21613
Loop:20064 / 21613
Loop:20065 / 21613
Loop:20066 / 21613
Loop:20067 / 21613
Loop:20068 / 21613
Loop:20069 / 21613
Loop:20070 / 21613
Loop:20071 / 21613
Loop:20072 / 21613
Loop:20073 / 21613
Loop:20074 / 21613
Loop:20075 /

Loop:20455 / 21613
Loop:20456 / 21613
Loop:20457 / 21613
Loop:20458 / 21613
Loop:20459 / 21613
Loop:20460 / 21613
Loop:20461 / 21613
Loop:20462 / 21613
Loop:20463 / 21613
Loop:20464 / 21613
Loop:20465 / 21613
Loop:20466 / 21613
Loop:20467 / 21613
Loop:20468 / 21613
Loop:20469 / 21613
Loop:20470 / 21613
Loop:20471 / 21613
Loop:20472 / 21613
Loop:20473 / 21613
Loop:20474 / 21613
Loop:20475 / 21613
Loop:20476 / 21613
Loop:20477 / 21613
Loop:20478 / 21613
Loop:20479 / 21613
Loop:20480 / 21613
Loop:20481 / 21613
Loop:20482 / 21613
Loop:20483 / 21613
Loop:20484 / 21613
Loop:20485 / 21613
Loop:20486 / 21613
Loop:20487 / 21613
Loop:20488 / 21613
Loop:20489 / 21613
Loop:20490 / 21613
Loop:20491 / 21613
Loop:20492 / 21613
Loop:20493 / 21613
Loop:20494 / 21613
Loop:20495 / 21613
Loop:20496 / 21613
Loop:20497 / 21613
Loop:20498 / 21613
Loop:20499 / 21613
Loop:20500 / 21613
Loop:20501 / 21613
Loop:20502 / 21613
Loop:20503 / 21613
Loop:20504 / 21613
Loop:20505 / 21613
Loop:20506 / 21613
Loop:20507 /

Loop:20887 / 21613
Loop:20888 / 21613
Loop:20889 / 21613
Loop:20890 / 21613
Loop:20891 / 21613
Loop:20892 / 21613
Loop:20893 / 21613
Loop:20894 / 21613
Loop:20895 / 21613
Loop:20896 / 21613
Loop:20897 / 21613
Loop:20898 / 21613
Loop:20899 / 21613
Loop:20900 / 21613
Loop:20901 / 21613
Loop:20902 / 21613
Loop:20903 / 21613
Loop:20904 / 21613
Loop:20905 / 21613
Loop:20906 / 21613
Loop:20907 / 21613
Loop:20908 / 21613
Loop:20909 / 21613
Loop:20910 / 21613
Loop:20911 / 21613
Loop:20912 / 21613
Loop:20913 / 21613
Loop:20914 / 21613
Loop:20915 / 21613
Loop:20916 / 21613
Loop:20917 / 21613
Loop:20918 / 21613
Loop:20919 / 21613
Loop:20920 / 21613
Loop:20921 / 21613
Loop:20922 / 21613
Loop:20923 / 21613
Loop:20924 / 21613
Loop:20925 / 21613
Loop:20926 / 21613
Loop:20927 / 21613
Loop:20928 / 21613
Loop:20929 / 21613
Loop:20930 / 21613
Loop:20931 / 21613
Loop:20932 / 21613
Loop:20933 / 21613
Loop:20934 / 21613
Loop:20935 / 21613
Loop:20936 / 21613
Loop:20937 / 21613
Loop:20938 / 21613
Loop:20939 /

Loop:21319 / 21613
Loop:21320 / 21613
Loop:21321 / 21613
Loop:21322 / 21613
Loop:21323 / 21613
Loop:21324 / 21613
Loop:21325 / 21613
Loop:21326 / 21613
Loop:21327 / 21613
Loop:21328 / 21613
Loop:21329 / 21613
Loop:21330 / 21613
Loop:21331 / 21613
Loop:21332 / 21613
Loop:21333 / 21613
Loop:21334 / 21613
Loop:21335 / 21613
Loop:21336 / 21613
Loop:21337 / 21613
Loop:21338 / 21613
Loop:21339 / 21613
Loop:21340 / 21613
Loop:21341 / 21613
Loop:21342 / 21613
Loop:21343 / 21613
Loop:21344 / 21613
Loop:21345 / 21613
Loop:21346 / 21613
Loop:21347 / 21613
Loop:21348 / 21613
Loop:21349 / 21613
Loop:21350 / 21613
Loop:21351 / 21613
Loop:21352 / 21613
Loop:21353 / 21613
Loop:21354 / 21613
Loop:21355 / 21613
Loop:21356 / 21613
Loop:21357 / 21613
Loop:21358 / 21613
Loop:21359 / 21613
Loop:21360 / 21613
Loop:21361 / 21613
Loop:21362 / 21613
Loop:21363 / 21613
Loop:21364 / 21613
Loop:21365 / 21613
Loop:21366 / 21613
Loop:21367 / 21613
Loop:21368 / 21613
Loop:21369 / 21613
Loop:21370 / 21613
Loop:21371 /

In [13]:
df.sample(5)

,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15,level,size,is_waterfront,year,year_week
8722,3333500096,2015-03-05,625000.00,4,3.00,2350,5627,1.00,0,0,5,8,1490,860,1960,0,98118,47.55,-122.27,2020,5627,2,2,no,2015,2015-09
13501,7524200180,2014-10-23,207000.00,4,2.00,1690,7728,1.50,0,0,4,7,1690,0,1967,0,98198,47.37,-122.32,1480,8009,0,1,no,2014,2014-42
20083,2143700756,2014-09-29,388000.00,4,2.50,2090,5040,2.00,0,0,3,9,2090,0,2014,0,98055,47.48,-122.23,1430,12000,1,2,no,2014,2014-39
5537,2726049169,2014-09-05,625000.00,3,1.75,1580,20588,1.00,0,0,3,8,1580,0,1970,0,98125,47.71,-122.29,2080,7800,2,1,no,2014,2014-35
8646,6133100120,2014-09-23,995000.00,3,2.50,2460,10300,1.00,0,0,3,10,2460,0,1992,0,98117,47.70,-122.39,2410,5250,3,2,no,2014,2014-38


In [26]:
df.to_csv('../files/new_dataset.csv')

# 4.0 - Adicione os seguintes filtros no mapa:
- Tamanho mínimo da área da sala estar. 
- Número mínimo de banheiros. 
- Valor máximo do Preço. 
- Tamanho máximo da área do porão. 
- Filtro das condições do imóvel. 
- Filtro por ano de construção.


In [12]:
df.head()

,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15,level,size
0,7129300520,20141013T000000,221900.00,3,1.00,1180,5650,1.00,0,0,3,7,1180,0,1955,0,98178,47.51,-122.26,1340,5650,0,0
1,6414100192,20141209T000000,538000.00,3,2.25,2570,7242,2.00,0,0,3,7,2170,400,1951,1991,98125,47.72,-122.32,1690,7639,2,3
2,5631500400,20150225T000000,180000.00,2,1.00,770,10000,1.00,0,0,3,6,770,0,1933,0,98028,47.74,-122.23,2720,8062,0,0
3,2487200875,20141209T000000,604000.00,4,3.00,1960,5000,1.00,0,0,5,7,1050,910,1965,0,98136,47.52,-122.39,1360,5000,2,2
4,1954400510,20150218T000000,510000.00,3,2.00,1680,8080,1.00,0,0,3,8,1680,0,1987,0,98074,47.62,-122.05,1800,7503,2,1


In [55]:
#Condition house
df.loc[df['condition'] <= 2,'condition_type'] = 'bad'
df.loc[df['condition'] == 3,'condition_type'] = 'regular'
df.loc[df['condition'] == 4,'condition_type'] = 'regular'
df.loc[df['condition'] == 5,'condition_type'] = 'good'

In [97]:
style = {'description_width': 'initial'}

#Botões interativos

size_room = widgets.IntSlider( 
    value = 290,
    min = 290, 
    max = 13540, 
    step = 1,
    description='Sala de Estar',
    disable=False,
    orientation='horizontal',
    style = style
)

qtd_bathroom = widgets.Dropdown(
    options=df['bathrooms'].sort_values().unique().tolist(),
    description='Banheiros',
    disabled=False,
    style=style
)


price_limit = widgets.IntSlider( 
    value = 540000,
    min = 75000, 
    max = 77000000, 
    step = 1,
    description='Preço Máximo',
    disable=False,
    style = style
)

area_basement = widgets.IntSlider( 
    value = 0,
    min = 0, 
    max = 4820, 
    step = 1,
    description='Área do Porão',
    disable=False,
    style = style
)

type_house = widgets.Dropdown(
    options=df['condition_type'].sort_values().unique().tolist(),
    description='Condição do imóvel',
    disabled=False,
    style=style
)


year_built = widgets.IntSlider( 
    value = 1900,
    min = 1900, 
    max = 2015, 
    step = 1,
    description='Ano de Construção',
    disable=False,
    style = style
)





In [98]:
def update_map( df,bathroom, condition, year, room, limit, basement):
    
    #Definindo as colunas para o mapa
    houses = df[(df['sqft_living'] >= room) &
                (df['bathrooms'] >= bathroom) &
                (df['price'] <= limit) & 
                (df['sqft_basement'] >= basement) & 
                (df['condition_type'] >= condition) & 
                (df['yr_built'] >= year)][['sqft_living','bathrooms','sqft_basement','condition','yr_built','id','price','lat','long','level']]

    #Criando mapa
    maps_price = px.scatter_mapbox(data_frame= houses,
                lat='lat',
                lon='long',
                hover_name='id',
                color='level',                  
                size='price',                  
                color_discrete_sequence=['darkblue'],
                color_continuous_scale=px.colors.sequential.Viridis,
                size_max=30,                  
                zoom=8,
                mapbox_style='open-street-map',
                height=1000) 

    #Definindo as margens do mapa
    maps_price.update_layout(margin={'r':0, 't':0, 'l':0, 'b':0})
    maps_price.show()

widgets.interactive( update_map, df=fixed(df), bathroom=qtd_bathroom,condition=type_house, year=year_built, room=size_room, limit=price_limit, basement=area_basement)

interactive(children=(Dropdown(description='Banheiros', options=(0.0, 0.5, 0.75, 1.0, 1.25, 1.5, 1.75, 2.0, 2.…

# 5.0 - Adicione os seguintes filtros no Dashboard:
- Filtro por data disponível para compra.
- Filtro por ano de renovação.
- Filtro se possui vista para a água ou não

In [110]:
data = df_raw.copy()
#Waterfront
data['is_waterfront'] = data['waterfront'].apply( lambda x: 'yes' if x == 1 else 'no' )

In [111]:
# mudando o formato da data
data['year'] = pd.to_datetime(data['date']).dt.strftime('%Y')
data['date'] = pd.to_datetime(data['date']).dt.strftime('%Y-%m-%d')
data['year_week'] = pd.to_datetime(data['date']).dt.strftime('%Y-%U')

# criando o botão para filtrar os dados
date_limit = widgets.Dropdown(
    options=df['date'].sort_values().unique(),
    value='2014-06-01',
    description='Disponível',
    disabled=False,
    style=style
)

date_renovated = widgets.Dropdown(
    options=df['yr_renovated'].sort_values().unique(),
    value=2014,
    description='Renovação',
    disabled=False,
    style=style
)

waterfront_bar = widgets.Dropdown( 
    options= df['is_waterfront'].unique().tolist(), 
    value='yes', 
    description='Vista para o mar', 
    disable=False)


In [112]:
def update_dash( data, date, renovated, waterfront):
    # Filter data
    df = data[(data['date'] <= date) & 
              (data['yr_renovated'] >= renovated) & 
              (data['is_waterfront'] == waterfront)]
               
    
    fig = plt.figure( figsize=(21,12))
    specs = gridspec.GridSpec( ncols=2, nrows=2, figure=fig )
    
    ax1 = fig.add_subplot( specs[0, :] ) # Primeira linha
    ax2 = fig.add_subplot( specs[1, 0] ) # Segunda Linha e Primeira coluna
    ax3 = fig.add_subplot( specs[1, 1] ) # Segunda Linha e Segunda coluna
    
    by_year = df[['id', 'year']].groupby( 'year').sum().reset_index()
    ax1.bar( by_year['year'], by_year['id'] )
    
    by_day = df[['id', 'date']].groupby( 'date').mean().reset_index()
    ax2.plot( by_day['date'], by_day['id'] )
    ax2.set_title( 'Preço por Dia' )
    
    by_week_of_year = df[['id', 'year_week']].groupby( 'year_week').mean().reset_index()
    ax3.bar( by_week_of_year['year_week'], by_week_of_year['id'] )
    ax3.set_title( 'Preço por semana do ano' )
    plt.xticks( rotation=60 );

In [113]:
widgets.interactive( update_dash, data=fixed( data ), date=date_limit, renovated=date_renovated, waterfront=waterfront_bar)

interactive(children=(Dropdown(description='Disponível', index=30, options=('2014-05-02', '2014-05-03', '2014-…

# 6.0 - Anotações da aula (While) 

In [15]:
import requests as r

i = 1
df_jobs = pd.DataFrame()

while True:
    
    print('page: {}'.format(i))

    url = 'https://jobs.github.com/positions.json?description=+Software+Engineer&location=&page={}'.format(i)
    response = r.request('GET',url)
    
    if response.json() != []:
        data = response.json()[0]
        df2 = pd.DataFrame(data,index=[0])
        
        print(df2[['id','type']])
        
        df_jobs = pd.concat([df_jobs,df2], axis=0)
        
        i = i + 1
        
    else:
        break

page: 1
                                     id       type
0  b0150533-8885-4407-8bb0-7de8377b6a5a  Full Time
page: 2
                                     id       type
0  51a1518e-d701-40bc-9496-146f56fa70a5  Full Time
page: 3
                                     id       type
0  c708e6b8-b786-4ee6-8c7a-0d93eb9c557a  Full Time
page: 4


In [16]:
df_jobs.head()

,id,type,url,created_at,company,company_url,location,title,description,how_to_apply,company_logo
0,b0150533-8885-4407-8bb0-7de8377b6a5a,Full Time,https://jobs.github.com/positions/b0150533-888...,Fri Feb 05 17:35:37 UTC 2021,10up Inc.,None,Remote,Senior Front End Engineer,<p>RESPONSIBILITIES</p>\n<ul>\n<li>Interpret d...,"<p><a href=""https://jobs.lever.co/10up-2/464e8...",https://jobs.github.com/rails/active_storage/b...
0,51a1518e-d701-40bc-9496-146f56fa70a5,Full Time,https://jobs.github.com/positions/51a1518e-d70...,Tue Jan 26 11:15:26 UTC 2021,Ignitia AB,https://www.ignitia.se,Sweden,Full Stack Developer,<p>Full Stack Developer</p>\n<p>Looking for ex...,"<p><a href=""https://apply.workable.com/ignitia...",https://jobs.github.com/rails/active_storage/b...
0,c708e6b8-b786-4ee6-8c7a-0d93eb9c557a,Full Time,https://jobs.github.com/positions/c708e6b8-b78...,Thu Jan 14 14:50:20 UTC 2021,COMPASS Pathways plc,https://compasspathways.com/,"London, UK or New York",DevOps Engineer,<p><strong>Job overview</strong></p>\n<p>We ar...,"<p>If interested, please send your CV and stat...",https://jobs.github.com/rails/active_storage/b...
